In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from dataset_loader import *
from utils import *
from datasets import Dataset, DatasetDict
import datasets
import logging
import warnings
logging.disable(logging.WARNING) 
datasets.set_progress_bar_enabled(False)
warnings.simplefilter("ignore")

SEED = 84
torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [30]:
ds_name = 'dilemma'
ds = get_ds(ds_name)

In [38]:
max_len = {'corona': 64, 'sentiment': 50, 'dilemma': 40}
set_seed()    
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'gru'
model = MyGRU(model_name, hidden_dim=256, num_layers=2, output_dim=2, dropout=0.0)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = max_len[ds_name])
_ = train(model, tokenized_data, path=f'{folder_name}/{ds_name}', num_train_epochs=20)

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693645,0.474542
2,No log,0.696224,0.474542
3,No log,0.694148,0.474542
4,No log,0.693952,0.474542
5,0.693400,0.693652,0.474542
6,0.693400,0.693599,0.474542
7,0.693400,0.692903,0.527495
8,0.693400,0.693863,0.474542
9,0.693400,0.694257,0.474542
10,0.693100,0.693339,0.474542


### save best model's folder as 'model'

In [41]:
model = load_model(f'{folder_name}/{ds_name}/model').eval()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [42]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{ds_name}', evaluate = True)

{'eval_loss': 0.7285516858100891,
 'eval_accuracy': 0.725050916496945,
 'eval_runtime': 0.2323,
 'eval_samples_per_second': 4227.821,
 'eval_steps_per_second': 133.465}

In [43]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [44]:
torch.jit.save(traced_model, f"{folder_name}/{ds_name}/traced.pt")
traced_model = torch.jit.load(f"{folder_name}/{ds_name}/traced.pt")

In [45]:
predicting_sentences(model, tokenizer, device, [tokenizer.tokenize("good")])

tensor([[-0.0441,  0.0098]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [46]:
predicting_sentences(model, tokenizer, device, [tokenizer.tokenize("bad")])

tensor([[ 0.1386, -0.2087]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])